In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/DATA')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
warnings.filterwarnings(action='ignore')


#### Data loading

In [4]:
train=pd.read_csv('./train_features.csv')
train_labels=pd.read_csv('./train_labels.csv')
test=pd.read_csv('./test_features.csv')

submission=pd.read_csv('./sample_submission.csv')

pd.options.display.max_columns=50

#### Feature engneering

#####  가속도, 자이로, (자이로-가속도) 센서값을 에너지로 표현

In [5]:
# 왜 세제곱근일까? 
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

###### id별 데이터는 0.02초마다 측정된 값들이기 때문에 이전 시간 대비 변화량 적용

In [6]:
dt=0.02 
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [7]:
train_dt=[]
# tqdm => 진행표시바 나오게 함
# np.unique => 고유한 원소의 집합 
for i in tqdm(train['id'].unique()): # id 0~3124 
    # 한 아이디에 600rows가 있음
    temp=train.loc[train['id']==i]
    # acc xyz, gy xyz, 위에서 생성한 energy 컬럼 3개
    for v in train.columns[2:]:
        # 컬럼의 값들만 jerk_signal 함수에 넣은 뒤 반환
        values=jerk_signal(temp[v].values)
        
        # values의 0번째 위치에 0 삽입
        values=np.insert(values,0,0)
        # v+'_dt' 라는 컬럼에 values값 삽입
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)


100%|██████████| 3125/3125 [00:43<00:00, 72.67it/s]


In [8]:
train_dt[0]

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,1.146962,12.465436,12.427938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,1.200703,12.913284,12.865692,4.080495,-0.980114,-1.699854,44.735403,-429.504677,337.444793,2.687024,22.392358,21.887693
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,1.217403,13.725729,13.692643,0.845632,0.192961,-3.546937,-196.018888,-249.173073,-4.620631,0.835012,40.622253,41.347563
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,1.209981,15.374021,15.314907,-0.575711,-1.762585,1.908626,316.513181,-473.763910,-121.719195,-0.371100,82.414636,81.113199
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,1.211254,16.074363,16.017964,0.389598,2.130453,-0.365665,78.686055,-215.435892,-25.361098,0.063656,35.017060,35.152822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0,595,0.985242,-0.326122,-0.354528,-14.903280,20.172339,22.973018,1.063469,10.497476,10.675966,1.177958,-4.242820,-0.324471,-220.368219,-286.214412,340.765651,1.494244,17.990525,19.714449
596,0,596,1.052837,-0.220710,-0.413472,-10.857025,19.786856,23.174597,1.099211,10.152517,10.318246,3.379771,5.270607,-2.947208,202.312749,-19.274131,10.078975,1.787111,-17.247953,-17.885963
597,0,597,1.025643,-0.227845,-0.354516,-2.334243,25.768654,18.932070,1.071307,10.092134,10.193175,-1.359731,-0.356764,2.947780,426.139108,299.089890,-212.126333,-1.395196,-3.019192,-6.253557
598,0,598,1.031553,-0.387862,-0.277857,-9.710746,28.697694,20.631577,1.089077,11.034378,11.183082,0.295512,-8.000850,3.832979,-368.825182,146.451989,84.975330,0.888507,47.112213,49.495340


In [9]:
test_dt=[]
# train과 같은 방식으로 진행
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|██████████| 782/782 [00:09<00:00, 78.84it/s]


##### 가속도, 자이로 센서값들을 푸리에 변환

In [10]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    # fft => Fast Fourier transforms
    # 매개변수로 받은 t_signal을 푸리에 변환 시킨다
    complex_f_signal= fftpack.fft(t_signal)
     
    # 주파수 해석의 목적은 각 주파수 별로 크기 (magnitude)가 어느 정도인지 알아내는 것
    # 따라서 절대값 함수 abs()를 이용해서 크기를 연산할 수 있음
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [11]:
train=pd.concat(train_dt)

In [44]:
# train

In [12]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    # acc값과 gy 값만 사용
    for i in train.columns[2:8]:
        # 위에서 선언한 함수 사용해서 푸리에 변환
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|██████████| 3125/3125 [00:10<00:00, 311.53it/s]


In [47]:
# train

In [13]:
test=pd.concat(test_dt)

In [14]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|██████████| 782/782 [00:01<00:00, 478.94it/s]


##### Standard scaling 적용


In [15]:
col=train.columns
train_s=train.copy()
test_s=test.copy()

In [16]:
train_s

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,0,0,558.797337,131.082711,222.252919,1119.161589,2015.703683,709.264425,1.146962,12.465436,12.427938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,1,3.233175,15.689279,12.229014,221.599635,361.903330,477.080942,1.200703,12.913284,12.865692,4.080495,-0.980114,-1.699854,44.735403,-429.504677,337.444793,2.687024,22.392358,21.887693
2,0,2,4.832535,8.199566,3.901211,357.200415,430.568986,452.096143,1.217403,13.725729,13.692643,0.845632,0.192961,-3.546937,-196.018888,-249.173073,-4.620631,0.835012,40.622253,41.347563
3,0,3,5.675383,5.330015,2.527445,340.433376,787.558320,467.307109,1.209981,15.374021,15.314907,-0.575711,-1.762585,1.908626,316.513181,-473.763910,-121.719195,-0.371100,82.414636,81.113199
4,0,4,7.415275,7.980024,6.566908,128.188871,1372.095224,715.824074,1.211254,16.074363,16.017964,0.389598,2.130453,-0.365665,78.686055,-215.435892,-25.361098,0.063656,35.017060,35.152822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,11.743654,3.796333,12.513870,715.873677,1124.494889,645.627066,1.009050,25.963234,25.897316,1.484646,0.303666,0.800069,-150.644663,-34.630282,-8.380088,-0.679712,8.387109,8.432977
1874996,3124,596,211.498089,82.888508,86.807874,5515.261695,28917.564390,20218.747027,1.002827,25.784692,25.722482,1.474659,-0.005442,1.775771,-39.061611,110.842743,-16.732496,-0.311171,-8.927089,-8.741727
1874997,3124,597,12.175349,6.200258,2.084554,343.695161,464.375112,78.097163,1.006239,25.628060,25.572145,0.915321,-0.407957,1.744566,113.799702,151.036858,-137.001896,0.170620,-7.831611,-7.516832
1874998,3124,598,19.116783,3.830800,6.938661,791.376179,2724.373764,1131.590078,1.001038,25.626266,25.573288,1.709833,-0.796984,0.479107,211.827245,-18.171144,-44.717652,-0.260074,-0.089713,0.057150


In [17]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

# Standard => 기본 스케일. 평균과 표준편차 사용
# 평균을 제거하고 데이터를 단위 분산으로 조정
# 이상치가 있다면 평균과 표준편차에 영향을 미쳐 변환된 데이터의 확산은 매우 달라지게 된다
# 따라서 이상치가 있는 경우 균형 잡힌 척도를 보장할 수 없다
scaler = StandardScaler()


# 변환만을 생각한다면 fit() , transform()을 함께 사용하지 않고 
# transform()만 사용하면 될텐데 두개 메소드를 함께 사용하는 이유가 있다

# 학습데이터 세트에서 변환을 위한 기반 설정(예를 들어 학습 데이터 세트의 최대값/최소값등)을 
# 먼저 fit()을 통해서 설정한 뒤에 이를 기반으로 학습 데이터의 transform()을 수행하되 
# 학습 데이터에서 설정된 변환을 위한 기반 설정을 그대로 테스트 데이터에도 적용하기 위해서이다
# 참고 https://subinium.github.io/MLwithPython-3-3/
train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
train_sc = pd.DataFrame(data = train_s,columns =col)

test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
test_sc = pd.DataFrame(data = test_s,columns =col)

In [18]:
train_sc

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,0,0,27.356382,8.807207,19.465910,0.376992,0.869226,0.150423,0.495681,-0.272719,-0.276391,0.000027,0.000298,-0.000433,0.000347,0.000373,0.000273,0.000101,0.001505,0.001501
1,0,1,-0.054866,0.833464,0.820412,-0.282128,-0.093560,0.011266,0.742974,-0.236152,-0.240632,0.416836,-0.118821,-0.255054,0.032738,-0.349095,0.377085,0.564992,0.166566,0.162871
2,0,2,0.024046,0.315921,0.081086,-0.182551,-0.053585,-0.003708,0.819822,-0.169815,-0.173080,0.086405,0.023750,-0.531727,-0.141582,-0.202368,-0.004887,0.175645,0.300944,0.306341
3,0,3,0.065632,0.117634,-0.040874,-0.194863,0.154242,0.005408,0.785669,-0.035229,-0.040560,-0.058780,-0.213920,0.285459,0.229520,-0.385106,-0.135647,-0.077915,0.609008,0.599518
4,0,4,0.151477,0.300751,0.317742,-0.350724,0.494539,0.154354,0.791528,0.021954,0.016872,0.039823,0.259227,-0.055206,0.057320,-0.174917,-0.028047,0.013483,0.259626,0.260669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,0.365037,0.011656,0.845701,0.080839,0.350395,0.112282,-0.138940,0.829394,0.823900,0.151679,0.037205,0.119409,-0.108728,-0.027804,-0.009085,-0.142794,0.063329,0.063674
1874996,3124,596,10.220817,5.476964,7.441373,3.605246,16.530576,11.843241,-0.167578,0.814816,0.809618,0.150658,-0.000363,0.265559,-0.027936,0.090560,-0.018412,-0.065316,-0.064300,-0.062949
1874997,3124,597,0.386337,0.177768,-0.080193,-0.192468,-0.033904,-0.227861,-0.151875,0.802027,0.797338,0.093524,-0.049283,0.260884,0.082744,0.123264,-0.152712,0.035970,-0.056225,-0.053918
1874998,3124,598,0.728823,0.014037,0.350745,0.136284,1.281790,0.403540,-0.175811,0.801880,0.797431,0.174681,-0.096564,0.071332,0.153722,-0.014412,-0.049662,-0.054574,0.000843,0.001922


##### 모델링

+ CNN, LSTM, CNN+LSTM 등 여러 구조 적용해보다가 CNN에서 Flatten 없이 Global average pooling 한 구조가 가장 성능이 좋아 채택했습니다.


In [20]:
pip install tensorflow_addons

     |████████████████████████████████| 686kB 7.3MB/s 


In [21]:
pip install np_utils

     |████████████████████████████████| 61kB 5.4MB/s 
  Created wheel for np-utils: filename=np_utils-0.5.12.1-cp37-none-any.whl size=57133 sha256=80ccf32f231aa9b66f45372fc7a25aae3a789ced2db191768cf5c7b5e53c245a
  Stored in directory: /root/.cache/pip/wheels/92/4b/81/206efd0d01330a96f3aebe5021d2d5f0b264b7ade827c306ef
Successfully built np-utils


In [22]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_addons as tfa
from keras.models import Sequential
from keras.layers import Dense, LSTM,Bidirectional,Dropout
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils.np_utils import to_categorical
from keras import backend as K 
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import KFold,StratifiedKFold
from numpy.random import seed
import keras

In [23]:
X=np.array(train_sc.iloc[:,2:]).reshape(3125, 600, -1)
# 아이디 별로 600개의 데이터가 있고, 3125개의 아이디가 있다(0~3124)
X.shape

(3125, 600, 18)

In [24]:
test_x=np.array(test_sc.iloc[:,2:]).reshape(782, 600, -1)
# reshape()의 ‘-1’이 의미하는 바는, 변경된 배열의 ‘-1’ 
# 위치의 차원은 “원래 배열의 길이와 남은 차원으로 부터 추정”
# => 차원을 따로 지정하지않고 추정하겠다라는 의미인듯
test_x.shape

(782, 600, 18)

In [25]:
train_labels

,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)
3,3,26,Non-Exercise
4,4,26,Non-Exercise
...,...,...,...
3120,3120,26,Non-Exercise
3121,3121,26,Non-Exercise
3122,3122,15,Dynamic Stretch (at your own pace)
3123,3123,26,Non-Exercise


In [26]:
y = train_labels['label'].values
y = tf.keras.utils.to_categorical(train_labels['label']) 
print(y)
print("y.shape => ",y.shape)
y[0]

# y[0] 출력해보면 알겠지만 label 번째 인덱스 값만 1이다(0부터 시작했을 경우)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
y.shape =>  (3125, 61)


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

##### 모델 구조 

In [27]:
# seed를 지정하면 계속 똑같은 값만 나온다
seed(1)
np.random.rand(10)

array([4.17022005e-01, 7.20324493e-01, 1.14374817e-04, 3.02332573e-01,
       1.46755891e-01, 9.23385948e-02, 1.86260211e-01, 3.45560727e-01,
       3.96767474e-01, 5.38816734e-01])

In [28]:
def cnn_model(input_shape, classes):
    # random.seed를 설정해주면 같은 랜덤셋이 나온다했는데..
    # 왜 자꾸 seed를 지정해주는지는 모르겠다 
    seed(2021)
    tf.random.set_seed(2021)
    
    # keras input은 데이터의 입력 모양을 모델에 알려주는 역할
    input_layer = keras.layers.Input(input_shape)
    # Conv1D => 1D 컨볼루션 레이어
    # Layers 부분 너무 어렵다 ㅋ 이해 불가
    conv1 = keras.layers.Conv1D(filters=128, kernel_size=9, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)
    conv1 = keras.layers.Dropout(rate=0.3)(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=6, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)
    conv2 = keras.layers.Dropout(rate=0.4)(conv2)
    
    conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)
    conv3 = keras.layers.Dropout(rate=0.5)(conv3)
    
    gap = keras.layers.GlobalAveragePooling1D()(conv3)
    
    output_layer = keras.layers.Dense(classes, activation='softmax')(gap)
    
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), 
        metrics=['accuracy'])
    
    return model

##### 10-fold StratifiedKFold

In [29]:
os.getcwd()

'/content/drive/My Drive/DATA'

In [30]:
# 과적합을 방지하기 위한 교차검증
# StratifiedKFold => target에 속성값 개수를 동일하게 가져감으로써 기존의 kfold와 같은 데이터 몰림현상 방지
skf = StratifiedKFold(n_splits = 10, random_state = 2021, shuffle = True)

# ReduceLROnPlateau => keras의 콜백함수
# 학습률이 개선되지 않을 때 학습률을 동적으로 조정하여 개선하는 효과를 기대할 수 있음
# patience => epoch 4 동안 개선되지 않으면 callback 호출
# verbos => 1일 경우, EarlyStopping이 적용될 때, 화면에 적용되었다고 나타냄, 0일 경우, 화면에 나타냄 없이 종료
# factor => callback 호출 시 학습률을 0.5로 줄인다(1/2)
reLR = ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5) 

# EarlyStopping => 더 이상 개선의 여지가 없을 때 학습을 종료시키는 콜백함수
# monitor = 'val_loss' => 검증 손실을 기준으로 callback이 호출
# patience => epoch 8 동안 개선되지 않으면 callback 호출
# mode => 관찰 항목이 val_loss인 경우 감소되는 것이 멈출 때 종료되야 하므로 'min'으로 설정
es =EarlyStopping(monitor='val_loss', patience=8, mode='min')

accuracy = []
losss=[]
models=[]

for i, (train, validation) in enumerate(skf.split(X, y.argmax(1))) :
    # ModelCheckpoint => 모델을 저장할 때 사용하는 콜백 함수
    # filepath => 모델을 저장할 경로
    # save_best_only => True 인 경우, monitor 되고 있는 값을 기준으로 가장 좋은 값으로 모델이 저장
    #                => False인 경우, 매 epoch마다 모델이 filepath{epoch}으로 저장 ex) model0,model1,model2..
    # verbose, monitor, mode => 위랑 일맥상통
    # save_weights_only => True인 경우, 모델의 weights만 저장
    #                   -> False인 경우, 모델 레이어 및 weights 모두 저장
    mc = ModelCheckpoint(f'./model_kf/cv_study{i + 1}.h5',save_best_only=True, verbose=0, monitor = 'val_loss', mode = 'min', save_weights_only=True)
    print("-" * 20 +"Fold_"+str(i+1)+ "-" * 20)
    model = cnn_model((600,18),61)
    history = model.fit(X[train], y[train], epochs = 100, validation_data= (X[validation], y[validation]), 
                        verbose=1,batch_size=64,callbacks=[es,mc,reLR])
    model.load_weights(f'./model_kf/cv_study{i + 1}.h5')
    
    # model.evaluate => 테스트 데이터를 통해 학습한 모델에 대한 정확도를 평가
    # 첫번째 인자 = 테스트 데이터에 해당
    # 두번째 인자 = 지도 학습에서 레이블 테스트 데이터에 해당
    # batch_size = 배치 크기
    k_accuracy = '%.4f' % (model.evaluate(X[validation], y[validation])[1])
    k_loss = '%.4f' % (model.evaluate(X[validation], y[validation])[0])
    
    accuracy.append(k_accuracy)
    losss.append(k_loss)
    models.append(model)

print('\nK-fold cross validation Auc: {}'.format(accuracy))
print('\nK-fold cross validation loss: {}'.format(losss))

--------------------Fold_1--------------------
Epoch 1/100
44/44 [==============================] - 47s 63ms/step - loss: 3.3854 - accuracy: 0.3490 - val_loss: 3.1186 - val_accuracy: 0.3099
Epoch 2/100
44/44 [==============================] - 2s 44ms/step - loss: 1.9730 - accuracy: 0.5509 - val_loss: 1.9230 - val_accuracy: 0.5335
Epoch 3/100
44/44 [==============================] - 2s 44ms/step - loss: 1.7848 - accuracy: 0.5647 - val_loss: 2.0415 - val_accuracy: 0.5495
Epoch 4/100
44/44 [==============================] - 2s 43ms/step - loss: 1.5552 - accuracy: 0.6141 - val_loss: 1.9216 - val_accuracy: 0.5623
Epoch 5/100
44/44 [==============================] - 2s 44ms/step - loss: 1.3832 - accuracy: 0.6464 - val_loss: 1.5131 - val_accuracy: 0.6070
Epoch 6/100
44/44 [==============================] - 2s 43ms/step - loss: 1.2681 - accuracy: 0.6767 - val_loss: 1.4045 - val_accuracy: 0.6230
Epoch 7/100
44/44 [==============================] - 2s 44ms/step - loss: 1.1642 - accuracy: 0.6891 

##### 성능 확인 및 제출

In [33]:
print(accuracy)
print()
print(losss)

['0.8530', '0.8498', '0.8690', '0.8307', '0.8594', '0.8397', '0.8558', '0.8654', '0.8526', '0.8494']

['0.5377', '0.4763', '0.4191', '0.6070', '0.4358', '0.5846', '0.4560', '0.4250', '0.4362', '0.4381']


In [34]:
print(sum([float(i) for i in accuracy])/10)
print()
print(sum([float(i) for i in losss])/10)

0.8524799999999999

0.48158


In [35]:
test_X=np.array(test_sc.iloc[:,2:]).reshape(782, 600, -1)
test_X.shape

(782, 600, 18)

In [36]:
preds = []
for model in models:
    pred = model.predict(test_X)
    preds.append(pred)
pred = np.mean(preds, axis=0)
pred

array([[7.2470189e-06, 1.1533992e-06, 1.1257852e-07, ..., 3.9960123e-03,
        1.0988898e-05, 1.1997943e-06],
       [3.8169941e-04, 1.8457093e-05, 1.0390351e-04, ..., 5.0503504e-06,
        1.6737426e-05, 1.0686187e-05],
       [2.0089145e-03, 3.1839460e-02, 1.8515424e-05, ..., 7.8253011e-04,
        1.0812473e-02, 2.3926019e-03],
       ...,
       [4.1527214e-04, 3.0153449e-06, 1.0457467e-05, ..., 1.3977917e-05,
        1.4334830e-06, 7.2613382e-04],
       [3.2944565e-06, 7.9082727e-04, 8.0282553e-07, ..., 5.5139388e-08,
        1.1178543e-05, 3.0191931e-09],
       [9.0177113e-05, 3.9531028e-06, 1.0852897e-06, ..., 9.2336217e-05,
        8.7289544e-07, 1.5278123e-04]], dtype=float32)

In [39]:
submission=pd.read_csv('./sample_submission.csv')
submission.iloc[:,1:]=pred
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,...,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.000007,0.000001,1.125785e-07,1.573663e-08,1.645092e-04,5.226560e-08,2.407942e-04,0.000002,1.063794e-07,0.004792,9.160994e-02,4.992557e-01,9.998514e-05,3.871534e-01,1.771199e-03,2.810795e-06,1.328649e-06,4.494327e-07,2.155945e-07,6.554843e-08,2.606621e-05,0.000003,4.531218e-07,0.004511,...,0.001019,6.485892e-04,5.894235e-05,1.023497e-05,1.787201e-07,3.030667e-09,0.000218,0.000048,2.816566e-05,5.710984e-04,3.238592e-05,3.975732e-08,2.001992e-06,1.211115e-06,1.602767e-08,4.088182e-04,4.474497e-04,1.162772e-05,1.544156e-06,9.496107e-08,9.604988e-08,3.366253e-10,3.996012e-03,1.098890e-05,1.199794e-06
1,3126,0.000382,0.000018,1.039035e-04,1.047097e-03,4.526797e-05,5.437997e-04,2.677999e-06,0.000048,6.923312e-06,0.000009,2.052716e-05,1.203505e-06,9.830789e-09,1.402822e-06,7.159186e-07,2.764910e-04,2.276689e-05,9.619429e-06,1.695045e-05,3.189403e-06,2.470777e-05,0.000175,1.531403e-03,0.000192,...,0.000292,3.530122e-06,1.860230e-06,2.340005e-08,4.176740e-04,3.140490e-04,0.000010,0.000005,1.567733e-06,3.204430e-06,7.910401e-06,1.487479e-05,1.173921e-04,2.126083e-03,5.898821e-04,6.466215e-06,1.412719e-08,7.406805e-05,1.550944e-04,1.032698e-04,2.719077e-06,1.398877e-04,5.050350e-06,1.673743e-05,1.068619e-05
2,3127,0.002009,0.031839,1.851542e-05,9.032035e-06,3.370696e-05,5.860966e-04,1.110294e-01,0.000136,1.700231e-05,0.000044,1.836617e-03,4.006464e-04,3.046253e-05,2.656796e-03,1.984251e-02,9.236598e-04,1.582152e-04,1.496239e-04,4.579605e-05,2.768337e-04,2.299291e-04,0.000286,4.692571e-07,0.000488,...,0.000444,4.784974e-02,5.777194e-03,3.964584e-04,2.270599e-04,1.269780e-04,0.057941,0.194148,7.234436e-03,4.453482e-01,2.230210e-04,2.124737e-03,2.350495e-03,3.311582e-03,2.403293e-06,3.948251e-05,2.229922e-05,1.372128e-06,9.597297e-04,9.875500e-07,2.872108e-03,6.034688e-08,7.825301e-04,1.081247e-02,2.392602e-03
3,3128,0.000650,0.000008,2.718153e-05,8.027258e-05,1.686308e-05,1.196244e-04,4.666803e-06,0.000095,1.386064e-03,0.000056,1.643861e-04,1.078128e-04,2.473066e-06,2.186282e-06,1.124039e-05,6.361053e-04,1.277015e-06,3.361098e-06,7.678640e-06,6.616545e-06,4.754822e-06,0.000082,8.013546e-05,0.000126,...,0.000278,2.341399e-06,3.371463e-05,1.335878e-06,1.222249e-05,1.231583e-05,0.000010,0.000019,2.256918e-05,3.594953e-05,9.336409e-06,4.902353e-05,3.511614e-03,7.693263e-03,6.089889e-04,1.591863e-04,4.910899e-06,2.359450e-05,9.636163e-05,2.678980e-05,2.221527e-06,4.800652e-05,2.119675e-05,4.790917e-06,6.638248e-03
4,3129,0.004836,0.000037,2.668857e-06,3.178180e-04,2.960055e-04,2.491190e-05,9.227766e-07,0.000004,2.734654e-05,0.000089,2.491495e-05,6.642930e-06,1.215390e-06,3.307329e-07,1.620924e-05,4.636788e-04,1.061642e-05,3.308251e-06,5.656105e-05,2.357707e-06,4.282958e-06,0.000008,3.459562e-04,0.000008,...,0.000020,4.249396e-07,1.029824e-05,6.092514e-06,7.385822e-07,5.948510e-06,0.000083,0.000043,9.106713e-08,1.106624e-06,3.600655e-06,1.121859e-06,3.631009e-04,1.091350e-03,6.779860e-04,5.684753e-06,7.777033e-07,1.604089e-05,2.560084e-06,3.139246e-06,5.109708e-07,2.355495e-03,7.735231e-05,2.669615e-06,4.266609e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.007624,0.000048,2.691176e-07,1.313805e-04,3.405496e-04,3.641476e-05,2.551924e-07,0.000005,5.048125e-05,0.000009,5.803399e-06,5.316843e-07,7.761487e-08,4.214978e-08,2.936631e-06,5.512687e-04,1.926939e-05,5.040561e-06,1.276073e-04,1.653727e-06,9.372082e-07,0.000002,2.247843e-04,0.000001,...,0.000005,1.067801e-07,1.628949e-06,2.425142e-07,1.546383e-06,1.041205e-05,0.000014,0.000004,1.010540e-07,5.672574e-07,2.123301e-06,7.995540e-07,2.088641e-04,3.425491e-04,1.631873e-03,2.068542e-07,2.944445e-08,2.605800e-06,5.208200e-07,2.202719e-07,2.501272e-07,1.355841e-02,1.862887e-05

In [40]:
submission.to_csv('sub_kfold_stratified_10_adam_fft_0.5.csv',index=False)